# Multicast GameServer

In [1]:
######## GAME ########

def game_start():
    
    global game
    
    response = requests.get("https://opentdb.com/api.php?amount=5")
    
    #Por si falla la API
    if (response.status_code != requests.codes.ok):
        sock.sendto(pickle.dumps((clientID, bytes_ErrorAPI)) , (MCAST_GRP, MCAST_PORT))
        print("Error:", response.status_code, response.text)
        game = False
    else:
        game = True
        
    
    if (game == False):
        print("API ERROR")
        
    else:
        
        for i in range(len(response.json()["results"])):
            
            correct_answ = generate_question(response, i)
            
            #For the generate_question sendto (En la primera iteración no lo tenemos en cuenta)
            if (i != 0):
                packed = sock.recv(1024)
            
            packed = sock.recv(4096) #Wait [BLOCK]
            data = pickle.loads(packed)
            
            
            print("\n\n\nEsta recibiendo: ", str(data[1]))
        
            
            #print(str(data[0]) + ": " + str(data[1]) + " DEBUG")

            if (str(data[1]) == "Quit"):
                client_leave(str(data[0]))
                packed = sock.recv(4096) #Wait [BLOCK]
                packed = sock.recv(4096) #Wait [BLOCK]
                data = pickle.loads(packed)
            
            check_answer(str(data[0]), str(data[1]), correct_answ)
                
            #For the check_answer sendto
            packed = sock.recv(1024)
     
            

        game_over()

In [2]:
####### Thread functions #######    
    
#Thread for idle state, waiting for game start
def thread_waiting():
    
    global game
    
    while game==False:
        
        try:
            msg = "Waiting for a game to start..."
            print(msg)
            sock.sendto(pickle.dumps((clientID, msg)) , (MCAST_GRP, MCAST_PORT))
            time.sleep(10)
        except(OSError):
            print("Socket closed")
            break
            
#Thread to recieve join messages while idle game
def thread_idle_recv():
    
    global game
    
    while game == False:
        
        try:
            packeduser = sock.recv(1024) #Wait [BLOCK]
            datauser = pickle.loads(packeduser)

            if (str(datauser[1]) == "JOIN"):
                joinmsg = str(datauser[0]) + " joined the game."
                print(joinmsg)
                correctas[str(datauser[0])] = 0  #Inicializamos user
                sock.sendto(pickle.dumps((clientID, joinmsg)) , (MCAST_GRP, MCAST_PORT))
        except(OSError):
            print("Socket closed")
            break

In [3]:
### FUNCTIONS

def generate_question(response, i):
    
    question = response.json()["results"][i]["question"]

    print("GameServer sending question: ", question)

    correct_answ = response.json()["results"][i]["correct_answer"]
    incorrect_answrs = response.json()["results"][i]["incorrect_answers"]

    incorrect_answrs.append(correct_answ)

    print("All answers: ", incorrect_answrs)

    #OJO este envía muchas mas cosas! (Añadimos la ID "SERVER-QUESTION" para que el cliente sepa indentificar cuando llega una pregunta del server  del resto de mensajes)
    pack = (clientID, "SERVER-QUESTION" , question, incorrect_answrs)
    send = pickle.dumps(pack)
    sock.sendto(send, (MCAST_GRP, MCAST_PORT))
    
    return correct_answ

def check_answer(clientid, clientanswer, correct_answ):

    if (clientanswer == correct_answ):
        #print("El usuario " + clientid + " acertó la pregunta.")
        msg = clientid + " envio la respuesta correcta, +1 punto"
        #sock.sendto(pickle.dumps((clientID, msg)), (MCAST_GRP, MCAST_PORT))
        correctas[clientid] +=1
    else:
        #print("El usuario " + clientid +  " envió una respuesta incorrecta.")
        msg = "El usuario " + clientid +  " envió una respuesta incorrecta, la respuesta correcta era: " + correct_answ
        #sock.sendto(pickle.dumps((clientID, msg)), (MCAST_GRP, MCAST_PORT))
        
    generate_ranking(correctas, msg)
    
def client_leave(clientid):
    
    msg = clientid + ": " + "Salió del juego"
    print(msg)
        
    data_pack = (clientID, msg)
    packed = pickle.dumps(data_pack)
    sock.sendto(packed, (MCAST_GRP, MCAST_PORT))
    
    #Le sacamos del diccionario de puntuación
    correctas.pop(clientid)
    
    
def generate_ranking(correctas_dict, agregatedmsg):
    dic_ordenado = sorted(correctas_dict.items(), key=lambda x: x[1], reverse=True)
    
    msgRanking = agregatedmsg + "\n" + "Ranking:\n"
    
    #sock.sendto(pickle.dumps((clientID, "Ranking:\n")), (MCAST_GRP, MCAST_PORT))
    for posicion, (clientid, puntuacion) in enumerate(dic_ordenado, start=1):
        msg = "#" + str(posicion) + " Usuario: " + clientid + " Puntos: " + str(puntuacion) + "\n"
        msgRanking += msg
    
    print (msgRanking)
    sock.sendto(pickle.dumps((clientID, msgRanking)), (MCAST_GRP, MCAST_PORT))
        
    return dic_ordenado
        
def reiniciar_puntuaciones():
    for user in correctas:
        correctas[user] = 0
    
def game_over():
    
    global game
    
    print("GAME OVER")
    endmsg = "GAME OVER"
    #sock.sendto(pickle.dumps((clientID, "GAME OVER")), (MCAST_GRP, MCAST_PORT))
    
    dic_ordenado = generate_ranking(correctas, endmsg)
    
    msg = "Winnner: " + str(next(iter(dic_ordenado)))
    print(msg)
    sock.sendto(pickle.dumps((clientID, msg)), (MCAST_GRP, MCAST_PORT))
    
    reiniciar_puntuaciones()

    #readerWait = threading.Thread(target=thread_waiting)
    #readerWait.start()

    #readerRecv = threading.Thread(target=thread_idle_recv)
    #readerRecv.start()
    
    game = False
    
    sock.sendto(pickle.dumps((clientID, "Se acabó el juego, gracias por jugar. Bye...")) , (MCAST_GRP, MCAST_PORT))
    sock.sendto(pickle.dumps((clientID, "END")) , (MCAST_GRP, MCAST_PORT)) #Para indicar a los users que el server se va OFF

    print("Se acabó el juego, gracias por jugar. Bye...")
    
    sock.close()
    

In [4]:
####### MAIN #######

import socket
import struct
import pickle
import threading
import time
import requests

MCAST_GRP = '224.1.1.1'
MCAST_PORT = 5007
clientID = 'GameServer'

bytes_ErrorAPI = str.encode("Error al generar una pregunta")

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)

#Para unirse al grupo de multicast
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

#Listen only on the socket
sock.bind(('', MCAST_PORT))

#Tell the OS to add the socket to the multicast group
mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

#Set TTL
ttl = struct.pack('b', 1)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, ttl)

#Format:
 # clientID: nº of correct answers
correctas = {}

#for game status handling
global game
game = False
    
########################## EJECUTAR TODAS LAS FUNCIONES ANTES DE LLEGAR A ESTA LINEA #####################################
    
#GAME IDLE
readerWait = threading.Thread(target=thread_waiting)
readerWait.start()

#HANDLE JOIN
readerRecv = threading.Thread(target=thread_idle_recv)
readerRecv.start()

########## MAIN #########

# Bucle while para permitir infinitas partidas seguidas
while True:
    
    #CMDS: START, END
    cmd = input("Introduce un comando: ")

    if (cmd == "START"):
        sock.sendto(pickle.dumps((clientID, "Una nueva partida esta a punto de empezar!")) , (MCAST_GRP, MCAST_PORT))
        #ime.sleep(5) #Para que les de tiempo a los users a prepararse
        game_start() #[BLOCK]
        
        break
        
    elif (cmd == "END"):
        sock.sendto(pickle.dumps((clientID, "Se acabó el juego, gracias por jugar. Bye...")) , (MCAST_GRP, MCAST_PORT))
        sock.sendto(pickle.dumps((clientID, "END")) , (MCAST_GRP, MCAST_PORT)) #Para indicar a los users que el server se va OFF
        
        print("Se acabó el juego, gracias por jugar. Bye...")
        
        #Para que los threads terminen
        game = True
        game = False
        
        sock.close()
        break
        
    else:
        print("Ese comando no existe, los comandos disponibles son: START (Iniciar una nueva partida) y END (Apagar el servidor y notificarlo a los usuarios conectados)")

Waiting for a game to start...
User 1 joined the game.
Introduce un comando: START
GameServer sending question:  Which character class in Dungeons and Dragons 5th edition gains it&#039;s powers from making a pact with a being of higher power?
All answers:  ['Wizard', 'Sorceror', 'Cleric', 'Warlock']



Esta recibiendo:  Wizard
El usuario User 1 envió una respuesta incorrecta, la respuesta correcta era: Warlock
Ranking:
#1 Usuario: User 1 Puntos: 0

GameServer sending question:  Gabe Newell was born in which year?
All answers:  ['1970', '1960', '1972', '1962 ']



Esta recibiendo:  1962 
User 1 envio la respuesta correcta, +1 punto
Ranking:
#1 Usuario: User 1 Puntos: 1

GameServer sending question:  Leonardo da Vinci&#039;s Mona Lisa does not have eyebrows.
All answers:  ['False', 'True']



Esta recibiendo:  True
User 1 envio la respuesta correcta, +1 punto
Ranking:
#1 Usuario: User 1 Puntos: 2

GameServer sending question:  Sciophobia is the fear of what?
All answers:  ['Eating', 'Bri